## Dati di input

In [1]:
!head -n 2 input/*

==> input/palermo_temperature_2008.csv <==
2008-01-01,00:00:00,13.7
2008-01-01,01:00:00,13.0

==> input/palermo_temperature_2009.csv <==
2009-01-01,00:00:00,11.0
2009-01-01,01:00:00,11.0

==> input/palermo_temperature_2010.csv <==
2010-01-01,00:00:00,17.0
2010-01-01,01:00:00,17.0

==> input/palermo_temperature_2011.csv <==
2011-01-01,00:00:00,10.3
2011-01-01,01:00:00,10.5

==> input/palermo_temperature_2012.csv <==
2012-01-01,00:00:00,10.0
2012-01-01,01:00:00,12.0

==> input/palermo_temperature_2013.csv <==
2013-01-01,00:00:00,8.0
2013-01-01,01:00:00,8.5

==> input/palermo_temperature_2014.csv <==
2014-01-01,00:00:00,15.0
2014-01-01,01:00:00,15.0

==> input/palermo_temperature_2015.csv <==
2015-01-01,00:00:00,6.7
2015-01-01,01:00:00,6.5

==> input/palermo_temperature_2016.csv <==
2016-01-01,01:00:00,10.0
2016-01-01,02:00:00,10.0

==> input/palermo_temperature_2017.csv <==
2017-01-01,00:00:00,9.5
2017-01-01,01:00:00,9.7


## Mapper

In [2]:
%%writefile temperature_mapper.py
#!/home/cloudera/anaconda3/bin/python
import sys
import os

if 'temp_max' in os.environ:
    temp_max = float(os.environ['temp_max'])
else:
    temp_max = 50.0

for line in sys.stdin:
    line = line.strip()
    
    date, time, temperature = line.split(',')
    year, month, day = date.split('-')
    temperature = float(temperature)
    
    if temperature <= temp_max:
        print(year, temperature, sep='\t')

Overwriting temperature_mapper.py


## Reducer

In [3]:
%%writefile temperature_reducer.py
#!/home/cloudera/anaconda3/bin/python
import sys

last_year = None
max_temp = float('-inf')

for line in sys.stdin:
    line = line.strip()
    
    year, temperature = line.split('\t')
    temperature = float(temperature)
    
    if last_year and last_year != year:
        print(last_year, max_temp, sep='\t')
        last_year = year
        max_temp = temperature
    else:
        max_temp = max(max_temp, temperature)
        last_year = year

if last_year:
    print(last_year, max_temp, sep='\t')

Overwriting temperature_reducer.py


## Prova in locale

In [4]:
!chmod +x temperature_mapper.py temperature_reducer.py

!ls -l *.py

-rwxrwxr-x 1 cloudera cloudera 415 Apr  1 15:51 temperature_mapper.py
-rwxrwxr-x 1 cloudera cloudera 506 Apr  1 15:51 temperature_reducer.py


In [5]:
!cat input/* | ./temperature_mapper.py | sort | ./temperature_reducer.py

2008	37.5
2009	35.0
2010	40.7
2011	35.0
2012	38.5
2013	33.5
2014	37.0
2015	36.3
2016	41.0
2017	38.0


## Job MapReduce

In [6]:
# Se necessario, copiamo i file di input su HDFS
# !hdfs dfs -put input/ temperature/input

!hdfs dfs -ls temperature

Found 2 items
drwxr-xr-x   - cloudera cloudera          0 2019-02-28 16:28 temperature/input
drwxr-xr-x   - cloudera cloudera          0 2019-04-01 13:10 temperature/output


#### Comando MapReduce

In [7]:
!hdfs dfs -rm -r -f temperature/output

!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files ./temperature_mapper.py,./temperature_reducer.py \
-cmdenv temp_max='35' \
-input temperature/input \
-output temperature/output \
-mapper ./temperature_mapper.py \
-reducer ./temperature_reducer.py \
-numReduceTasks 3

Deleted temperature/output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.13.0.jar] /tmp/streamjob4496838716226312990.jar tmpDir=null
19/04/01 15:51:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/01 15:51:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/01 15:51:25 INFO mapred.FileInputFormat: Total input paths to process : 10
19/04/01 15:51:25 INFO mapreduce.JobSubmitter: number of splits:10
19/04/01 15:51:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1551366833984_0053
19/04/01 15:51:26 INFO impl.YarnClientImpl: Submitted application application_1551366833984_0053
19/04/01 15:51:26 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1551366833984_0053/
19/04/01 15:51:26 INFO mapreduce.Job: Running job: job_1551366833984_0053
19/04/01 15:51:31 INFO mapreduce.Job: Job job_1551366833984_0053 running in uber mode : false
19/04/01 15:51:31 INFO m

#### Visualizzazione risultati

Il risultato del job MapReduce è diverso da quello lanciato in locale, perché le treshold per la temperatura massima sono diverse (35°C e 50°C, rispettivamente)

In [8]:
!hdfs dfs -ls temperature/output

print('\nResults:')
!hdfs dfs -cat temperature/output/*

Found 4 items
-rw-r--r--   1 cloudera cloudera          0 2019-04-01 15:51 temperature/output/_SUCCESS
-rw-r--r--   1 cloudera cloudera         30 2019-04-01 15:51 temperature/output/part-00000
-rw-r--r--   1 cloudera cloudera         30 2019-04-01 15:51 temperature/output/part-00001
-rw-r--r--   1 cloudera cloudera         40 2019-04-01 15:51 temperature/output/part-00002

Results:
2009	35.0
2012	35.0
2015	34.5
2010	34.5
2013	33.5
2016	34.3
2008	35.0
2011	35.0
2014	35.0
2017	35.0
